In [123]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import simplejson as json
import requests
import time
from IPython.display import display


In [124]:
URI = "dbbikes2.cytgvbje9wgu.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbbikes2"
USER = "admin"
PASSWORD = "DublinBikes1"

In [125]:
engine = create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [126]:
sql = """
CREATE DATABASE IF NOT EXISTS dbbikes1;
"""
engine.execute(sql)

2023-02-22 15:34:34,200 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-02-22 15:34:34,200 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-22 15:34:34,404 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-02-22 15:34:34,404 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-22 15:34:34,528 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-02-22 15:34:34,528 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-22 15:34:34,721 INFO sqlalchemy.engine.Engine 
CREATE DATABASE IF NOT EXISTS dbbikes1;

2023-02-22 15:34:34,721 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-22 15:34:34,819 INFO sqlalchemy.engine.Engine COMMIT


In [127]:
for res in engine.execute("SHOW VARIABLES;"):
    print(res)

2023-02-22 15:34:35,107 INFO sqlalchemy.engine.Engine SHOW VARIABLES;
2023-02-22 15:34:35,107 INFO sqlalchemy.engine.Engine [raw sql] ()
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('back_log', '63')
('basedir', '/rdsdbbin/mysql-8.0.28.R4/')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_direct_non_transactional_updates', 'OFF')
('binlog_encryption', 'OFF')
('binlog_error_action', 'ABORT_SERVER')
('binlog_expire_logs_seconds', '2592000')

In [128]:
sql = """
CREATE TABLE IF NOT EXISTS station (
   address VARCHAR(256),
   banking INTEGER,
   bike_stands INTEGER,
   bonus INTEGER,
   contract_name VARCHAR(256),
   name VARCHAR(256),
   number INTEGER,
   position_lat REAL,
   position_lng REAL,
   status VARCHAR(256),
   timestamp INTEGER
)
"""
try:
    res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-22 15:34:35,448 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS station
2023-02-22 15:34:35,448 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-22 15:34:35,730 INFO sqlalchemy.engine.Engine COMMIT
2023-02-22 15:34:35,932 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS station (
   address VARCHAR(256),
   banking INTEGER,
   bike_stands INTEGER,
   bonus INTEGER,
   contract_name VARCHAR(256),
   name VARCHAR(256),
   number INTEGER,
   position_lat REAL,
   position_lng REAL,
   status VARCHAR(256),
   timestamp INTEGER
)

2023-02-22 15:34:35,932 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-22 15:34:36,168 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [129]:
sql = """
CREATE TABLE IF NOT EXISTS availability (
number INTEGER,
available_bikes INTEGER,
available_bike_stands INTEGER,
last_update INTEGER
)
"""
try:
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-22 15:34:36,399 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS availability (
number INTEGER,
available_bikes INTEGER,
available_bike_stands INTEGER,
last_update INTEGER
)

2023-02-22 15:34:36,405 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-22 15:34:36,580 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [130]:
#!/usr/bin/env python
import requests
import traceback
import datetime
import time
APIKEY="5ebe2af971884baaf7d3d2197368197fb873dd55"
NAME = "Dublin"
STATIONS = "https://api.jcdecaux.com/vls/v1/stations"

def write_to_file(text):
    with open("data/bikes_{}".format(now).replace(" ", "_"), "w") as f:
        f.write(r.text)

#def write_to_db(text):
    
def main():
    while True:
        try:
            now = datetime.datetime.now()
            r = requests.get(STATIONS, params={"apiKey":APIKEY, "contract":NAME})
            print(r,now)
            write_to_file(r.text)
            write_to_db(r.text)
            time.sleep(5*60)
        except:
            print(traceback.format_exc())
            #if engine is None:
                
    return
                

In [131]:
from datetime import datetime
now = datetime.now()#datetime
print(now)

timestamp = datetime.timestamp(now) #int
print(timestamp)

dt_object= datetime.fromtimestamp(timestamp) #object
print(type(dt_object))
print(dt_object)

2023-02-22 15:34:36.847227
1677080076.847227
<class 'datetime.datetime'>
2023-02-22 15:34:36.847227


In [132]:
def stations_to_db(text):
    stations = json.loads(text)
    #print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get('address'), int(station.get('banking')), station.get('bike_stands'), int(station.get('bonus')), station.get('contract_name'), station.get('name'), station.get('number'), station.get('position').get('lat'), station.get('position').get('lng'), station.get("status"), datetime.timestamp(datetime.now()) )
        engine.execute("insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)
        
    return

#JCKEY = "8ad0fc88de299d032d91bc99f1e01c34a44d39a0"
#NAME = "Dublin"
URI =  "https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey=8ad0fc88de299d032d91bc99f1e01c34a44d39a0"
r = requests.get(URI, params= {"api_key":JCKEY, "contract": NAME})
#json.loads(r.text)
stations_to_db(r.text)

{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 27, 'available_bikes': 3, 'status': 'OPEN', 'last_update': 1677079866000}
2023-02-22 15:34:37,033 INFO sqlalchemy.engine.Engine insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-22 15:34:37,041 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield North', 0, 30, 0, 'dublin', 'SMITHFIELD NORTH', 42, 53.349562, -6.278198, 'OPEN', 1677080077.033523)
2023-02-22 15:34:37,130 INFO sqlalchemy.engine.Engine COMMIT
{'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 16, 'available_bikes': 4, 'status': 'OPEN', 'last_update': 1677080028000}
2023-02-22 15:34:

In [133]:
import pandas as pd
sql_DF = pd.read_sql("SELECT * FROM station", con=engine, parse_dates=['activity_date'])

2023-02-22 15:35:13,102 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-02-22 15:35:13,106 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ('dbbikes2', 'SELECT * FROM station')
2023-02-22 15:35:13,309 INFO sqlalchemy.engine.Engine SELECT * FROM station
2023-02-22 15:35:13,309 INFO sqlalchemy.engine.Engine [raw sql] ()
